In [196]:
import numpy as np

import pandas as pd

from nltk.tokenize import word_tokenize

import re

import nltk

from contractions import contractions_dict

from nltk.corpus import stopwords

from spacy.lang.en.stop_words import STOP_WORDS

from itertools import filterfalse

from nltk import pos_tag

from nltk.stem import WordNetLemmatizer

from nltk.corpus import wordnet

from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.feature_extraction.text import CountVectorizer

from sklearn.feature_extraction.text import TfidfTransformer

from sklearn.decomposition import PCA

from sklearn.datasets import load_files

from sklearn.naive_bayes import GaussianNB

from sklearn.metrics._classification import classification_report

#nltk.download('averaged_perceptron_tagger')

#nltk.download('punkt')
#nltk.download('stopwords')

In [152]:
data_load = load_files('mini_newsgroups',encoding='latin1',decode_error='ignore')

In [153]:
data = pd.DataFrame({
    'text': data_load.data,
    'target': data_load.target,
    'category': [data_load.target_names[i] for i in data_load.target]
})

In [154]:
data

,text,target,category
0,Path: cantaloupe.srv.cs.cmu.edu!crabapple.srv....,4,comp.sys.mac.hardware
1,Newsgroups: sci.crypt\nPath: cantaloupe.srv.cs...,11,sci.crypt
2,"Newsgroups: sci.crypt,alt.privacy.clipper\nPat...",11,sci.crypt
3,Newsgroups: rec.autos\nPath: cantaloupe.srv.cs...,7,rec.autos
4,Xref: cantaloupe.srv.cs.cmu.edu soc.culture.ar...,17,talk.politics.mideast
...,...,...,...
1995,Xref: cantaloupe.srv.cs.cmu.edu rec.motorcycle...,8,rec.motorcycles
1996,Path: cantaloupe.srv.cs.cmu.edu!crabapple.srv....,12,sci.electronics
1997,Newsgroups: talk.politics.guns\nPath: cantalou...,16,talk.politics.guns
1998,Path: cantaloupe.srv.cs.cmu.edu!rochester!udel...,5,comp.windows.x


In [155]:
data_load.target_names

['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

In [156]:
data['category'].unique()

array(['comp.sys.mac.hardware', 'sci.crypt', 'rec.autos',
       'talk.politics.mideast', 'soc.religion.christian', 'sci.med',
       'talk.politics.misc', 'alt.atheism', 'sci.electronics',
       'comp.os.ms-windows.misc', 'comp.graphics', 'talk.religion.misc',
       'rec.sport.hockey', 'comp.windows.x', 'rec.sport.baseball',
       'talk.politics.guns', 'comp.sys.ibm.pc.hardware',
       'rec.motorcycles', 'sci.space', 'misc.forsale'], dtype=object)

In [157]:
data['text'][0]

"Path: cantaloupe.srv.cs.cmu.edu!crabapple.srv.cs.cmu.edu!fs7.ece.cmu.edu!europa.eng.gtefsd.com!howland.reston.ans.net!usc!cs.utexas.edu!uunet!pipex!marble.uknet.ac.uk!uknet!acorn!camcon!aw\nFrom: aw@camcon.co.uk (Alain Waha)\nNewsgroups: comp.sys.mac.hardware\nSubject: Re: New Duo Dock info.\nMessage-ID: <30003@io.camcon.co.uk>\nDate: 6 Apr 93 08:55:26 GMT\nReferences: <nazario-040493023201@stiles-42-kstar-node.net.yale.edu> <1993Apr4.164142.14744@umiami.ir.miami.edu> <nazario-050493004102@stiles-42-kstar-node.net.yale.edu>\nOrganization: Cambridge Consultants Ltd., Cambridge, UK\nLines: 7\n\n>> In article <nazario-040493023201@stiles-42-kstar-node.net.yale.edu>, nazario@pop.cis.yale.edu (Edgardo Nazario) writes:\n>> > The info I am about to give is not a rumour, it's the truth. The new\n>> > macintosh coming in the second quarter, will have a cpu of their own. \n\nExcuse me but... have not all Macs got a CPU!!!\n\nAlain\n"

In [158]:
data['category'].value_counts()

category
comp.sys.mac.hardware       100
sci.crypt                   100
sci.space                   100
rec.motorcycles             100
comp.sys.ibm.pc.hardware    100
talk.politics.guns          100
rec.sport.baseball          100
comp.windows.x              100
rec.sport.hockey            100
talk.religion.misc          100
comp.graphics               100
comp.os.ms-windows.misc     100
sci.electronics             100
alt.atheism                 100
talk.politics.misc          100
sci.med                     100
soc.religion.christian      100
talk.politics.mideast       100
rec.autos                   100
misc.forsale                100
Name: count, dtype: int64

In [159]:
data.isna().sum()

text        0
target      0
category    0
dtype: int64

In [160]:
data.tail(50)

,text,target,category
1950,Newsgroups: sci.electronics\nPath: cantaloupe....,12,sci.electronics
1951,Path: cantaloupe.srv.cs.cmu.edu!crabapple.srv....,13,sci.med
1952,Newsgroups: comp.sys.mac.hardware\nPath: canta...,4,comp.sys.mac.hardware
1953,Path: cantaloupe.srv.cs.cmu.edu!das-news.harva...,9,rec.sport.baseball
1954,Xref: cantaloupe.srv.cs.cmu.edu comp.org.eff.t...,11,sci.crypt
1955,Newsgroups: misc.forsale\nPath: cantaloupe.srv...,6,misc.forsale
1956,Newsgroups: rec.autos\nPath: cantaloupe.srv.cs...,7,rec.autos
1957,Xref: cantaloupe.srv.cs.cmu.edu alt.atheism:53...,19,talk.religion.misc
1958,Xref: cantaloupe.srv.cs.cmu.edu rec.audio.car:...,12,sci.electronics
1959,Xref: cantaloupe.srv.cs.cmu.edu alt.atheism:54...,0,alt.atheism


In [161]:
data.duplicated().sum()

0

In [162]:
data['text'].iloc[5]

"Path: cantaloupe.srv.cs.cmu.edu!rochester!udel!gatech!howland.reston.ans.net!agate!doc.ic.ac.uk!warwick!qmw-dcs!qmw!demon!gtoal.com!gtoal\nNewsgroups: sci.crypt\nFrom: gtoal@gtoal.com (Graham Toal)\nSubject: Re: Off the shelf cheap DES keyseach machine (Was: Re: Corporate acceptance of the wiretap chip)\nDate: Wed, 21 Apr 1993 10:24:28 +0000\nMessage-ID: <9304211024.AA21842@pizzabox.demon.co.uk>\nSender: usenet@demon.co.uk\nLines: 16\n\n: At the company I worked for previously, I received a file that was des encryped\n: and the person that had sent it, went on vaction.  Rather than wait two weeks\n: I set up a straight frontal attack with one key at a time.  It only took two(2)\n: days to crack the file.  No, I don't have any faith in DES.\n\n: A.G.\n\nFun!  Three questions:\n\n1) What hardware did you use?\n2) When you found the key, was there anything about it that was special\n   that meant you had been lucky to find it early? (Like the first 30 bits\n   all being 0)\n3) ... or did

In [163]:
def title_striping(text):
    lines = text.split('\n')
    for i, line in enumerate(lines):
        if line.strip() == '':
            return '\n'.join(lines[i+1:]).strip()
    return text.strip() 

In [164]:
data['text'] = data['text'].apply(lambda x: title_striping(x))

In [165]:
data.head()

,text,target,category
0,>> In article <nazario-040493023201@stiles-42-...,4,comp.sys.mac.hardware
1,Deeply grateful for citations to any papers on...,11,sci.crypt
2,"The system, or 'family', key would appear to b...",11,sci.crypt
3,In <C5Dq2n.5uB@news.cso.uiuc.edu> mwbg9715@uxa...,7,rec.autos
4,In article <2BDAD779.24910@news.service.uci.ed...,17,talk.politics.mideast


In [166]:
data['text'] = data['text'].apply(lambda x: word_tokenize(x))

In [167]:
data

,text,target,category
0,"[>, >, In, article, <, nazario-040493023201, @...",4,comp.sys.mac.hardware
1,"[Deeply, grateful, for, citations, to, any, pa...",11,sci.crypt
2,"[The, system, ,, or, 'family, ', ,, key, would...",11,sci.crypt
3,"[In, <, C5Dq2n.5uB, @, news.cso.uiuc.edu, >, m...",7,rec.autos
4,"[In, article, <, 2BDAD779.24910, @, news.servi...",17,talk.politics.mideast
...,...,...,...
1995,"[Anyone, interesting, in, a, mailing, list, fo...",8,rec.motorcycles
1996,"[Does, anyone, know, what, causes, the, ever-g...",12,sci.electronics
1997,"[In, article, <, 1993Apr23.120935.21848, @, ic...",16,talk.politics.guns
1998,"[Is, there, any, FAQ, list, for, Programming, ...",5,comp.windows.x


In [168]:
def token_normalizer(tokens_list):
    return map(lambda x: x.lower(),tokens_list)

In [169]:
data['text'] = data['text'].apply(lambda x: token_normalizer(x))

In [170]:
data['text'] = data['text'].apply(lambda x: list(x))

In [171]:
data

,text,target,category
0,"[>, >, in, article, <, nazario-040493023201, @...",4,comp.sys.mac.hardware
1,"[deeply, grateful, for, citations, to, any, pa...",11,sci.crypt
2,"[the, system, ,, or, 'family, ', ,, key, would...",11,sci.crypt
3,"[in, <, c5dq2n.5ub, @, news.cso.uiuc.edu, >, m...",7,rec.autos
4,"[in, article, <, 2bdad779.24910, @, news.servi...",17,talk.politics.mideast
...,...,...,...
1995,"[anyone, interesting, in, a, mailing, list, fo...",8,rec.motorcycles
1996,"[does, anyone, know, what, causes, the, ever-g...",12,sci.electronics
1997,"[in, article, <, 1993apr23.120935.21848, @, ic...",16,talk.politics.guns
1998,"[is, there, any, faq, list, for, programming, ...",5,comp.windows.x


In [172]:
def word_expansion_contracted(token):
    if token in contractions_dict.keys():
        return contractions_dict[token]
    else:
        return token

In [173]:
def contraction_expansion(tokens_list):
    return map(word_expansion_contracted,tokens_list)

In [174]:
data['text'] = data['text'].apply(lambda x: contraction_expansion(x))

In [175]:
data['text'] = data['text'].apply(lambda x: list(x))

In [176]:
data

,text,target,category
0,"[>, >, in, article, <, nazario-040493023201, @...",4,comp.sys.mac.hardware
1,"[deeply, grateful, for, citations, to, any, pa...",11,sci.crypt
2,"[the, system, ,, or, 'family, ', ,, key, would...",11,sci.crypt
3,"[in, <, c5dq2n.5ub, @, news.cso.uiuc.edu, >, m...",7,rec.autos
4,"[in, article, <, 2bdad779.24910, @, news.servi...",17,talk.politics.mideast
...,...,...,...
1995,"[anyone, interesting, in, a, mailing, list, fo...",8,rec.motorcycles
1996,"[does, anyone, know, what, causes, the, ever-g...",12,sci.electronics
1997,"[in, article, <, 1993apr23.120935.21848, @, ic...",16,talk.politics.guns
1998,"[is, there, any, faq, list, for, programming, ...",5,comp.windows.x


In [177]:
regex = r'^@[a-zA-z0-9]|^#[a-zA-Z0-9]|\w+:\/{2}[\d\w-]+(\.[\d\w-]+)*(?:(?:\/[^\s/]*))*|\W+|\d+|<("[^"]*"|\'[^\']*\'|[^\'">])*>|_+|[^\u0000-\u007f]+'

In [178]:
def check_waste_words(token):
    return re.search(regex,token)

In [179]:
def filter_waste_words(tokens_list):
    return filterfalse(check_waste_words,tokens_list)

In [180]:
data['text'] = data['text'].apply(lambda x: filter_waste_words(x))

In [181]:
data['text'] = data['text'].apply(lambda x: list(x))

In [182]:
data

,text,target,category
0,"[in, article, nazario, edgardo, nazario, write...",4,comp.sys.mac.hardware
1,"[deeply, grateful, for, citations, to, any, pa...",11,sci.crypt
2,"[the, system, or, key, would, appear, to, be, ...",11,sci.crypt
3,"[in, mark, wayne, blunier, writes, jimf, jim, ...",7,rec.autos
4,"[in, article, tclock, tim, clock, writes, in, ...",17,talk.politics.mideast
...,...,...,...
1995,"[anyone, interesting, in, a, mailing, list, fo...",8,rec.motorcycles
1996,"[does, anyone, know, what, causes, the, black,...",12,sci.electronics
1997,"[in, article, kdw, kenneth, whitehead, writes,...",16,talk.politics.guns
1998,"[is, there, any, faq, list, for, programming, ...",5,comp.windows.x


In [183]:
def split(tokens_list):
    return map(lambda x: re.split(regex,x)[0],tokens_list)

In [184]:
data['text'] = data['text'].apply(lambda x: split(x))

In [185]:
data['text'] = data['text'].apply(lambda x: list(x))

In [186]:
data['text'][0]

['in',
 'article',
 'nazario',
 'edgardo',
 'nazario',
 'writes',
 'the',
 'info',
 'i',
 'am',
 'about',
 'to',
 'give',
 'is',
 'not',
 'a',
 'rumour',
 'it',
 'the',
 'truth',
 'the',
 'new',
 'macintosh',
 'coming',
 'in',
 'the',
 'second',
 'quarter',
 'will',
 'have',
 'a',
 'cpu',
 'of',
 'their',
 'own',
 'excuse',
 'me',
 'but',
 'have',
 'not',
 'all',
 'macs',
 'got',
 'a',
 'cpu',
 'alain']

In [187]:
eng_stop_words = list(set(stopwords.words('english')).union(set(STOP_WORDS)))

In [188]:
len(eng_stop_words)

401

In [189]:
def is_valid_token(token):
    return not(token in eng_stop_words or re.search(r'\b\w\b|[^\u0000-\u007f]+|_+|\W+',token))

In [190]:
def stopword_removal(tokens_list):
    return filter(is_valid_token,tokens_list)

In [191]:
data['text'] = data['text'].apply(lambda x: stopword_removal(x))

In [192]:
data['text'] = data['text'].apply(lambda x: list(x))

In [193]:
data

,text,target,category
0,"[article, nazario, edgardo, nazario, writes, i...",4,comp.sys.mac.hardware
1,"[deeply, grateful, citations, papers, electron...",11,sci.crypt
2,"[system, key, appear, cryptographically, usele...",11,sci.crypt
3,"[mark, wayne, blunier, writes, jimf, jim, fros...",7,rec.autos
4,"[article, tclock, tim, clock, writes, article,...",17,talk.politics.mideast
...,...,...,...
1995,"[interesting, mailing, list, bikes, lifestyle,...",8,rec.motorcycles
1996,"[know, causes, black, border, edges, computer,...",12,sci.electronics
1997,"[article, kdw, kenneth, whitehead, writes, day...",16,talk.politics.guns
1998,"[faq, list, programming, windows, thankx, info...",5,comp.windows.x


In [194]:
data['text'][0]

['article',
 'nazario',
 'edgardo',
 'nazario',
 'writes',
 'info',
 'rumour',
 'truth',
 'new',
 'macintosh',
 'coming',
 'second',
 'quarter',
 'cpu',
 'excuse',
 'macs',
 'got',
 'cpu',
 'alain']